# DonorsChoose.org Application Screening

## Read in data

In [1]:
import os
import pandas as pd

# Set up directories
data_dir = 'data'
output_dir = 'output'
if not os.path.isdir(output_dir):
    os.mkdir(output_dir)

# Read in all available data
res = pd.read_csv(os.path.join(data_dir, 'resources.csv'), dtype='str')
train = pd.read_csv(os.path.join(data_dir, 'train.csv'), parse_dates=[4], dtype='str')
test = pd.read_csv(os.path.join(data_dir, 'test.csv'), parse_dates=[4], dtype='str')

## Comb through each feature

#### resources.csv

In [2]:
res.head(5)

,id,description,quantity,price
0,p233245,LC652 - Lakeshore Double-Space Mobile Drying Rack,1,149.00
1,p069063,Bouncy Bands for Desks (Blue support pipes),3,14.95
2,p069063,Cory Stories: A Kid's Book About Living With Adhd,1,8.45
3,p069063,"Dixon Ticonderoga Wood-Cased #2 HB Pencils, Bo...",2,13.59
4,p069063,EDUCATIONAL INSIGHTS FLUORESCENT LIGHT FILTERS...,3,24.95


In [3]:
# Convert numeric values to float
res['quantity'] = pd.to_numeric(res['quantity'], errors='coerce')
res['price'] = pd.to_numeric(res['price'], errors='coerce')

In [4]:
res.describe(include='all')

,id,description,quantity,price
count,1541272,1540980,1.541272e+06,1.541272e+06
unique,260115,332928,NaN,NaN
top,p137603,Apple - iPad� mini 2 with Wi-Fi - 16GB - Space...,NaN,NaN
freq,100,3037,NaN,NaN
mean,NaN,NaN,2.860509e+00,5.028398e+01
std,NaN,NaN,7.570345e+00,1.447326e+02
min,NaN,NaN,1.000000e+00,0.000000e+00
25%,NaN,NaN,1.000000e+00,7.900000e+00
50%,NaN,NaN,1.000000e+00,1.499000e+01
75%,NaN,NaN,2.000000e+00,3.980000e+01


Each proposal has a list of items requested. Each proposal may request multiple item types, and multiples of any given item. The quantity requested and price of each item is given. We will want to map the requests to a single proposal, so some summarizing will be necessary. There are many features possible here, it makes sense to anchor the desired features to answerable questions.

- **Do cheap or expensive proposals tend to get approved?**
    - Highest/Lowest price of items (max/min of `price`)
    - Total cost of request (sum of `price` * `quantity` for each proposal)
- **Do proposals with few or many items tend to get approved?**
    - Total number of items requested (sum of `quantity`)
    - Total number of unique items requested (count of items per proposal)
- **Does the type of item requested affect approval?**
    - Aggregation of all words from item descriptions (string join of `description`)

The possibility of information being gleaned from the list of descriptions means that text analysis will need to be used. This was obvious from the data descriptions but here it becomes apparent. 

#### train.csv

In [5]:
train.head(5)

,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved
0,p036502,484aaf11257089a66cfedc9461c6bd0a,Ms.,NV,2016-11-18 14:45:59,Grades PreK-2,Literacy & Language,Literacy,Super Sight Word Centers,Most of my kindergarten students come from low...,I currently have a differentiated sight word c...,NaN,NaN,My students need 6 Ipod Nano's to create and d...,26,1
1,p039565,df72a3ba8089423fa8a94be88060f6ed,Mrs.,GA,2017-04-26 15:57:28,Grades 3-5,"Music & The Arts, Health & Sports","Performing Arts, Team Sports",Keep Calm and Dance On,Our elementary school is a culturally rich sch...,We strive to provide our diverse population of...,NaN,NaN,My students need matching shirts to wear for d...,1,0
2,p233823,a9b876a9252e08a55e3d894150f75ba3,Ms.,UT,2017-01-01 22:57:44,Grades 3-5,"Math & Science, Literacy & Language","Applied Sciences, Literature & Writing",Lets 3Doodle to Learn,Hello;\r\nMy name is Mrs. Brotherton. I teach ...,We are looking to add some 3Doodler to our cla...,NaN,NaN,My students need the 3doodler. We are an SEM s...,5,1
3,p185307,525fdbb6ec7f538a48beebaa0a51b24f,Mr.,NC,2016-08-12 15:42:11,Grades 3-5,Health & Sports,Health & Wellness,"\""Kid Inspired\"" Equipment to Increase Activit...",My students are the greatest students but are ...,"The student's project which is totally \""kid-i...",NaN,NaN,My students need balls and other activity equi...,16,0
4,p013780,a63b5547a7239eae4c1872670848e61a,Mr.,CA,2016-08-06 09:09:11,Grades 6-8,Health & Sports,Health & Wellness,We need clean water for our culinary arts class!,My students are athletes and students who are ...,For some reason in our kitchen the water comes...,NaN,NaN,My students need a water filtration system for...,42,1


In [6]:
# Convert numeric values to float
train['teacher_number_of_previously_posted_projects'] = pd.to_numeric(train['teacher_number_of_previously_posted_projects'], errors='coerce')

In [7]:
train.describe(include='all')

,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved
count,182080,182080,182076,182080,182080,182080,182080,182080,182080,182080,182080,6374,6374,182080,182080.000000,182080
unique,182080,104414,5,51,180439,4,51,407,164282,147689,180984,6359,6336,179730,NaN,2
top,p012847,fa2f220b537e8653fb48878ebb38044d,Mrs.,CA,2016-09-01 00:00:03,Grades PreK-2,Literacy & Language,Literacy,Flexible Seating,As a teacher in a low-income/high poverty scho...,Students will be using Chromebooks to increase...,We will use the wobble chairs to provide non-t...,"Having taught engineering in college, I have c...",My students need electronic tablets to do all ...,NaN,1
freq,1,74,95405,25695,30,73890,39257,15775,377,46,24,2,3,84,NaN,154346
first,NaN,NaN,NaN,NaN,2016-04-27 00:03:38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,2017-04-30 23:50:43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.237055,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.016086,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN


Notes on first pass of features

- `id` - unique id of the project application
    - Used to tie `resources.csv` data in
- `teacher_id` - id of the teacher submitting the application
    - Counts of submissions per teacher captured in other feature, `teacher_number_of_previously_posted_projects`
    - Will be disregarded
- `teacher_prefix` - title of the teacher's name (Ms., Mr., etc.)
    - Categorical data with few unique values, likely useful
    - Needs to be encoded
- `school_state` - US state of the teacher's school
    - Categorical data with ~50 unique values, likely useful
    - Needs to be encoded
- `project_submitted_datetime` - application submission timestamp
    - Needs to be split into multiple columns
        - Year
        - Month
        - Day of year
        - Day of week
        - Hour of day
- `project_grade_category` - school grade levels (PreK-2, 3-5, 6-8, and 9-12)
    - Categorical data with few unique values, likely useful
    - Needs to be encoded
- `project_subject_categories` - category of the project (e.g., "Music & The Arts")
    - Categorical data with few unique values, likely useful
    - Several records have 2 entries, needs to be split into 2 new columns
    - Needs to be encoded
- `project_subject_subcategories` - sub-category of the project (e.g., "Visual Arts")
    - Categorical data with few unique values, likely useful
    - Several records have 2 entries, needs to be split into 2 new columns
    - Needs to be encoded
- `project_title` - title of the project
    - Needs to be normalized
- `project_essay_1` - `project_essay_4`
    - Essays will need to be combined, see section XYZ #FIXME
- `project_resource_summary` - summary of the resources needed for the project
    - Needs to be augmented with `resources.csv` data
- `teacher_number_of_previously_posted_projects` - number of previously posted applications by the submitting teacher
    - Useful as is
- `project_is_approved` - whether DonorsChoose proposal was accepted (0="rejected", 1="accepted"); train.csv only
    - Class labels

Note: Prior to May 17, 2016, the prompts for the essays were as follows:

- `project_essay_1`: "Introduce us to your classroom"
- `project_essay_2`: "Tell us more about your students"
- `project_essay_3`: "Describe how your students will use the materials you're requesting"
- `project_essay_4`: "Close by sharing why your project will make a difference"

The target variable is contained in `project_is_approved`, with binary classes: rejected=0 and approved=1. 

Looking at the training data it becomes apparent that two approaches should be used to solve this classification problem:

1. **Classification based on categorical features**
1. **Classification based on text information**

The questions and corresponding features that come to mind are:

- **Do the categorical features independently have an affect on approval?**
    - `['teacher_prefix', 'school_state', 'project_grade_category', 'project_subject_categories', 'project_subject_subcategories']`
    - Acceptance rate of each feature
    - Number of submissions of each feature
- **Does the date of submission affect approval?**
    - Approval rates by year, month, day of year, day of week, hour of day
- **Do new or experienced submitters get approved more?**
    - Acceptance rate of `teacher_number_of_previously_posted_projects`
- **Does the title affect approval?**
    - Number of words in `project_title`
    - Sentiment of `project_title`
- **The big one: Do the essays affect approval?**
    - Length of each essay
        - Number of characters
        - Number of words
        - Word density
    - After cleaning essay text* evaluate the following:
        - SVM linear classification
        - Topic models of each essay
        - doc2vec models of each essay
- **Does the resource summary affect approval?**
    - Closely linked with aggregated description field from resources table
    
*Cleaning text covered in section XYZ #FIXME

Essays are an interesting chunk of this dataset and potentially have a wealth of information in them. Every application has questions 1 and 2, but the counts of questions 3 and 4 are significantly lower. Typically if there are large amounts of missing data in an attribute, it would be thrown out. However, I believe that removing `project_essay_3` and `project_essay_4` from consideration would throw away useful data. According to the data description: 
>Note: Prior to May 17, 2016, the prompts for the essays were as follows:
- `project_essay_1`: "Introduce us to your classroom"
- `project_essay_2`: "Tell us more about your students"
- `project_essay_3`: "Describe how your students will use the materials you're requesting"
- `project_essay_4`: "Close by sharing why your project will make a difference"

>Starting on May 17, 2016, the number of essays was reduced from 4 to 2, and the prompts for the first 2 essays were changed to the following:
- `project_essay_1`: "Describe your students: What makes your students special? Specific details about their background, your neighborhood, and your school are all helpful."
- `project_essay_2`: "About your project: How will these materials make a difference in your students' learning and improve their school lives?"

To me, the newer question 1 appears to be a combination of the old questions 1 and 2, while the newer question 2 appears to be a combination of the old questions 3 and 4. The old questions 1 and 2 ask about the classroom and students, while the new question 2 asks about the students and the school. The old questions 3 and 4 ask about how materials will be used and how they will make a difference, while the new question 2 addresses the same topics.

As such I will combine the old question 1 and 2 into question 1, and the old question 3 and 4 into quesiton 2.

#### test.csv

In [8]:
test.head(5)

,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects
0,p233245,5724a0c3ce11008366fff36dab4b943c,Ms.,CA,2016-04-27 13:45:41,Grades PreK-2,Music & The Arts,Visual Arts,Art Will Make You Happy!,My 2nd grade students are amazing! They are v...,My class is made up of 12 boys and 12 girls. ...,My second grade class will really benefit from...,The genorous donations to my project will make...,My students need a drying rack for their art p...,2
1,p096795,445619941dc7cbe81c7be109dc61a56a,Mrs.,SC,2016-04-28 12:43:56,Grades 3-5,"Literacy & Language, Math & Science","Literature & Writing, Mathematics",Keeping up with the TIMEs,Students within the classroom work in small gr...,My students are all very talented young indivi...,"We do a lot of small group, where the student ...",This project is very important to my classroom...,My students need Time Magazines for next year ...,1
2,p236235,e92a4902b1611a189643d6f12c51e6a0,Mrs.,SC,2016-04-29 21:16:05,Grades PreK-2,Math & Science,"Applied Sciences, Mathematics",Building Bridges to Problem Solving,My students share a love of learning. These s...,My class consists of 14 energetic learners. O...,These Fairy Tale Problem Solving STEM kits wil...,These materials will be help my students with ...,My students need to be mentally stimulated to ...,0
3,p233680,8e92622d2985d3faac1de71609c4be72,Mrs.,IA,2016-04-27 22:32:43,Grades PreK-2,Literacy & Language,Literacy,Classroom Library,Reading is the gateway to the soul. Guiding c...,First graders enter the classroom each day rea...,Book bins will help to organize our classroom ...,"When students begin the first grade, many are ...",My students need an organized classroom library.,0
4,p171879,91a3c89981f626d9a0d067c65fb186ce,Mr.,CA,2016-04-27 18:59:15,Grades 6-8,"Music & The Arts, Special Needs","Performing Arts, Special Needs",Reeds so we can Read,"\""Mr. Reyes! I need another reed!\"" I hear t...",We have a diverse population with almost entir...,Each day in my class students consume supplies...,My students come from very troubled homes and ...,My students need reeds to perform in class eac...,1


In [9]:
# Convert numeric values to float
test['teacher_number_of_previously_posted_projects'] = pd.to_numeric(test['teacher_number_of_previously_posted_projects'], errors='coerce')

In [10]:
test.describe(include='all')

,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects
count,78035,78035,78034,78035,78035,78035,78035,78035,78035,78035,78035,2704,2704,78035,78035.000000
unique,78035,55508,5,51,77646,4,51,390,72755,69435,77826,2702,2692,77531,NaN
top,p048258,df8a4b7ad173b57f7ac52e447cc24043,Mrs.,CA,2016-09-01 00:00:06,Grades PreK-2,Literacy & Language,Literacy,Flexible Seating,I teach a special day class that is filled wit...,Students will be using Chromebooks to increase...,"We don't need a behemoth, expensive desktop co...","Currently, there is a huge problem with inner-...",My students need electronic tablets to do all ...,NaN
freq,1,37,40982,10723,8,31471,16777,6859,167,23,11,2,3,32,NaN
first,NaN,NaN,NaN,NaN,2016-04-27 00:06:07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,2017-04-30 23:37:50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.275082
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.981717
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000


Confirmed that the test data is of the same form as the training data, minus the class labels.

## Feature engineering

In [11]:
# Calculate the total cost of each item
res['cost'] = res['quantity'] * res['price']
# Calculate the total cost of each proposal
res_unique = res.groupby(by='id').sum()

In [12]:
# Split up subject categories into individual columns
train['subject_a'], train['subject_b'] = train['project_subject_categories'].str.split(',', 1).str
train['subject_c'], train['subject_d'] = train['project_subject_subcategories'].str.split(',', 1).str

# Split up datetime into columns
train['date_year'] = train['project_submitted_datetime'].dt.year
train['date_month'] = train['project_submitted_datetime'].dt.month
train['date_dow'] = train['project_submitted_datetime'].dt.dayofweek
train['date_doy'] = train['project_submitted_datetime'].dt.dayofyear
train['date_hour'] = train['project_submitted_datetime'].dt.hour

In [13]:
# Combine subject categories into single feature
cols = ['subject_a', 'subject_b', 'subject_c', 'subject_d']
# Remove escaped characters from text
for c in cols:
    train[c].fillna('', inplace=True)
train['subject_agg'] = train[cols].apply(' '.join, axis=1)

In [14]:
# Combine essays and project title into single feature
cols = ['project_title', 'project_essay_1', 'project_essay_2', 'project_essay_3', 'project_essay_4']
# Remove escaped characters from text
for c in cols:
    train[c].fillna('', inplace=True)
    train[c] = train[c].apply(lambda s: s.replace('\\"', ' '))
    train[c] = train[c].apply(lambda s: s.replace('\\r', ' '))
    train[c] = train[c].apply(lambda s: s.replace('\\n', ' '))
train['essay_agg'] = train[['project_title', 'project_essay_1', 'project_essay_2', 'project_essay_3', 'project_essay_4']].apply(' '.join, axis=1)

In [15]:
# Split out categorical features, text, and labels
text = train[['project_title',
              'project_essay_1',
              'project_essay_2',
              'project_essay_3',
              'project_essay_4',
              'project_resource_summary',
              'essay_agg',
              'subject_agg'
             ]]
categorical = train[['teacher_prefix',
                     'school_state',
                     'project_grade_category',
                     'subject_a',
                     'subject_b',
                     'subject_c',
                     'subject_d'
                    ]]
remaining = train[['id',
                   'teacher_number_of_previously_posted_projects',
                   'date_year',
                   'date_month',
                   'date_dow',
                   'date_doy',
                   'date_hour'
                  ]]
labels = train['project_is_approved']

In [16]:
encoded = categorical.apply(lambda s: s.astype('category').cat.codes)
cat_one_hot = pd.get_dummies(categorical, drop_first=True)

In [17]:
processed = pd.merge(cat_one_hot, remaining, how='left', left_index=True, right_index=True)
processed = pd.merge(processed, res_unique, how='left', left_on='id', right_on='id')
processed.drop('id', axis=1, inplace=True)

In [18]:
processed.head(5)

,teacher_prefix_Mr.,teacher_prefix_Mrs.,teacher_prefix_Ms.,teacher_prefix_Teacher,school_state_AL,school_state_AR,school_state_AZ,school_state_CA,school_state_CO,school_state_CT,...,"subject_d_ Warmth, Care & Hunger",teacher_number_of_previously_posted_projects,date_year,date_month,date_dow,date_doy,date_hour,quantity,price,cost
0,0,0,1,0,0,0,0,0,0,0,...,0,26,2016,11,4,323,14,6,299.98,899.94
1,0,1,0,0,0,0,0,0,0,0,...,0,1,2017,4,2,116,15,20,20.00,400.00
2,0,0,1,0,0,0,0,0,0,0,...,0,5,2017,1,6,1,22,1,469.99,469.99
3,1,0,0,0,0,0,0,0,0,0,...,0,16,2016,8,4,225,15,5,684.47,684.47
4,1,0,0,0,0,0,0,1,0,0,...,0,42,2016,8,5,219,9,2,355.50,711.00


## Categorical prediction (cross validation testing)

In [19]:
from sklearn import model_selection
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB

names = [#'Nearest Neighbors',
         #'Gaussian Process',
         'Decision Tree',
         'Random Forest',
         'Neural Net',
         #'Naive Bayes'
        ]
classifiers = [#KNeighborsClassifier(3),
               #GaussianProcessClassifier(1.0 * RBF(1.0)),
               DecisionTreeClassifier(max_depth=5),
               RandomForestClassifier(max_depth=5, n_estimators=10),
               MLPClassifier(alpha=1),
               #GaussianNB()
              ]

In [20]:
kfold = model_selection.KFold(n_splits=5, random_state=1138)

for name, clf in zip(names, classifiers):
    out = model_selection.cross_val_score(clf, processed, labels, cv=kfold)
    print('{} ACC: {:0.3f}%  STD: {:0.3f}%'.format(name, out.mean()*100.0, out.std()*100.0))

Decision Tree ACC: 84.765%  STD: 0.129%
Random Forest ACC: 84.768%  STD: 0.131%
Neural Net ACC: 82.259%  STD: 4.684%


## NLP Prediction (cross validation testing)

In [21]:
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

In [ ]:
stop_words = stopwords.words('english') + list(string.punctuation)
stm = PorterStemmer()
def normalize_text(s):
    tokens = [stm.stem(t.lower()) for t in word_tokenize(s) if t not in stop_words]
    normalized = ' '.join(tokens)
    return normalized
    
text['essay_norm'] = text['essay_agg'].apply(normalize_text)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

In [ ]:
kfold = model_selection.KFold(n_splits=5, random_state=1138)
vct = TfidfVectorizer()
vct_train = vct.fit_transform(text['essay_norm'])
clf = LinearSVC()

out = model_selection.cross_val_score(clf, vct_train, labels, cv=kfold)
print('ACC: {:0.3f}%  STD: {:0.3f}%'.format(out.mean()*100.0, out.std()*100.0))

## Predict output with test data #FIXME Functionalize everything so it can be done on train and test

In [ ]:
clf = RandomForestClassifier(max_depth=5, n_estimators=10)
clf.fit(rf_train, labels)
out_rf = clf.predict(rf_test)

In [ ]:
vct = TfidfVectorizer()
vct_train = vct.fit_transform(text['essay_norm'])
vct_test = vct.transform(test['essay_norm'])

clf = LinearSVC()
clf.fit(vct_train, labels)
out_nlp = clf.predict(vct_test)